In [1]:
password = '13310625'
system_commands = "apt-get update && apt-get upgrade -y && apt-get install -y gcc libfftw3-dev"

!echo {password} | sudo -S sh -c "{system_commands}"

!pip install numpy scipy


import numpy as np
import scipy.signal
import ctypes
import time

Hit:1 http://archive.ubuntu.com/ubuntu noble InRelease
Hit:2 https://repo.mongodb.org/apt/ubuntu noble/mongodb-org/8.2 InRelease
Hit:3 http://archive.ubuntu.com/ubuntu noble-updates InRelease
Hit:4 http://security.ubuntu.com/ubuntu noble-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu noble-backports InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Calculating upgrade... Done
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
gcc is already the newest version (4:13.2.0-7ubuntu1).
libfftw3-dev is already the newest version (3.3.10-1ubuntu3).
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.


In [2]:
def AutoCorr1_original(X):
    """Versión original usando la convolución de SciPy."""
    N = len(X)
    # La autocorrelación es la convolución de la señal con su inversa
    fft_cor = scipy.signal.fftconvolve(X, X[::-1], mode='full')[N-1:]
    # Normalización para obtener un estimador no sesgado
    fft_cor /= (N - np.arange(N))
    return fft_cor[:len(X)//2]

def AutoCorr1_optimized(X):
    """Versión optimizada usando la FFT de NumPy directamente."""
    N = len(X)
    # Calcular el tamaño óptimo para la FFT (potencia de 2)
    Nfft = 2 ** int(np.ceil(np.log2(2*N - 1)))
    Xf = np.fft.fft(X, Nfft)
    # Teorema de Wiener-Khinchin: Rxx = IFFT(|FFT(X)|^2)
    Rxx = np.fft.ifft(Xf * np.conjugate(Xf)).real
    # Normalización y selección de la primera mitad de los lags
    Rxx = Rxx[:N] / (N - np.arange(N))
    return Rxx[:len(X)//2]

def AutoCorr1_superfast(X):
    """Versión superrápida normalizada por el lag cero."""
    Xf = np.array(X, dtype=np.float32)
    N = Xf.size
    # Tamaño de la FFT usando operación de bits (más rápido)
    Nfft = 1 << (int(np.ceil(np.log2((N << 1) - 1))))
    F = np.fft.fft(Xf, Nfft)
    Rxx = np.fft.ifft(F * np.conjugate(F)).real[:N]
    # Normalización por Rxx[0] para que el máximo sea 1
    Rxx /= Rxx[0]
    return Rxx[:len(X)//2]

try:
    lib_c = ctypes.CDLL('./libautocorr.so')

    # Definir los tipos de los argumentos para las funciones C
    arg_types = [
        np.ctypeslib.ndpointer(dtype=np.float32, flags='C_CONTIGUOUS'),
        ctypes.c_int,
        np.ctypeslib.ndpointer(dtype=np.float32, flags='C_CONTIGUOUS')
    ]

    # Asignar tipos a cada función
    lib_c.autocorr_original_c.argtypes = arg_types
    lib_c.autocorr_original_c.restype = None
    lib_c.autocorr_optimized_c.argtypes = arg_types
    lib_c.autocorr_optimized_c.restype = None
    lib_c.autocorr_superfast_c.argtypes = arg_types
    lib_c.autocorr_superfast_c.restype = None

    # Wrappers de Python para llamar a las funciones C de forma sencilla
    def AutoCorr1_C_Original(X):
        n = len(X)
        x_c = np.array(X, dtype=np.float32)
        rxx_out_c = np.empty(n // 2, dtype=np.float32)
        lib_c.autocorr_original_c(x_c, n, rxx_out_c)
        return rxx_out_c

    def AutoCorr1_C_Optimized(X):
        n = len(X)
        x_c = np.array(X, dtype=np.float32)
        rxx_out_c = np.empty(n // 2, dtype=np.float32)
        lib_c.autocorr_optimized_c(x_c, n, rxx_out_c)
        return rxx_out_c

    def AutoCorr1_C_Superfast(X):
        n = len(X)
        x_c = np.array(X, dtype=np.float32)
        rxx_out_c = np.empty(n // 2, dtype=np.float32)
        lib_c.autocorr_superfast_c(x_c, n, rxx_out_c)
        return rxx_out_c

    c_library_loaded = True
except OSError:
    print("Advertencia: No se pudo cargar la librería C 'libautocorr.so'.")
    print("Las funciones C no estarán disponibles.")
    c_library_loaded = False


# -----------------------------------------------------------------------------
# 4. BUCLE DE BENCHMARK
# -----------------------------------------------------------------------------

# Diccionario con las funciones a probar
functions_to_test = {
    'Python Original': AutoCorr1_original,
    'Python Optimized': AutoCorr1_optimized,
    'Python Superfast': AutoCorr1_superfast
}
if c_library_loaded:
    functions_to_test.update({
        'C Original': AutoCorr1_C_Original,
        'C Optimized': AutoCorr1_C_Optimized,
        'C Superfast': AutoCorr1_C_Superfast
    })

sizes = [2**4, 2**6, 2**8, 2**10, 2**12, 2**14, 2**16, 2**18]
num_trials = 100
rng = np.random.default_rng()
methods = list(functions_to_test.keys())

mean_times = {m: [] for m in methods}
std_times = {m: [] for m in methods}

print(f"Ejecutando benchmark con {num_trials} pruebas por tamaño...")
for N in sizes:
    print(f"  Probando con tamaño N = {N}...")
    times_dict = {m: [] for m in methods}
    # Generar todas las señales de una vez para ser eficientes
    signals = rng.standard_normal((num_trials, N)).astype(np.float32)

    for x in signals:
        for name, func in functions_to_test.items():
            start = time.perf_counter_ns()
            _ = func(x)
            end = time.perf_counter_ns()
            times_dict[name].append((end - start) * 1e-9) # Convertir ns a s

    for m in methods:
        mean_times[m].append(np.mean(times_dict[m]))
        std_times[m].append(np.std(times_dict[m]))
print("Benchmark finalizado.\n")


# -----------------------------------------------------------------------------
# 5. PRUEBA CON UNA ÚNICA SEÑAL
# -----------------------------------------------------------------------------

# --- Señal de prueba: onda cuadrada con ruido ---
fs = 1000  # Hz
T = 2.0    # seconds
t = np.arange(0, T, 1/fs)
square_wave = np.sign(np.sin(2 * np.pi * 5 * t))
signal = square_wave + 0.2 * np.random.randn(len(t))

# --- Calcular autocorrelación y tiempo para una ejecución ---
print("\n--- Tiempos para una sola ejecución con señal de prueba ---\n")
results_single_run = {}
for name, func in functions_to_test.items():
    start_time = time.perf_counter()
    r = func(signal)
    elapsed = time.perf_counter() - start_time
    results_single_run[name] = (r, elapsed)
    print(f"{name}: máximo Rxx = {r.max():.3f}, tiempo = {elapsed*1e3:.2f} ms")

Ejecutando benchmark con 100 pruebas por tamaño...
  Probando con tamaño N = 16...
  Probando con tamaño N = 64...
  Probando con tamaño N = 256...
  Probando con tamaño N = 1024...
  Probando con tamaño N = 4096...
  Probando con tamaño N = 16384...
  Probando con tamaño N = 65536...
  Probando con tamaño N = 262144...
Benchmark finalizado.


--- Tiempos para una sola ejecución con señal de prueba ---

Python Original: máximo Rxx = 1.039, tiempo = 0.47 ms
Python Optimized: máximo Rxx = 1.039, tiempo = 0.22 ms
Python Superfast: máximo Rxx = 1.000, tiempo = 0.15 ms
C Original: máximo Rxx = 1.039, tiempo = 0.17 ms
C Optimized: máximo Rxx = 1.039, tiempo = 0.06 ms
C Superfast: máximo Rxx = 1.000, tiempo = 0.06 ms
